In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
from IPython.core.debugger import set_trace

# Import packages

In [83]:
from __future__ import print_function
import os
import sys
import inspect
import time
from datetime import datetime
import warnings, logging
import argparse
from easydict import EasyDict as edict
# 
import pickle
# 
import numpy as np
# Pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
# from torchsummary import summary
import torchmodel_summary
# Metrics
from sklearn.metrics import accuracy_score, log_loss

# Configs.py

In [150]:
todaystr = datetime.today().strftime('%Y%m%d')       
config = edict()
# MODEl related
config.MODEL = edict()
config.MODEL.NAME = 'DNN_model'
config.MODEL.INIT_WEIGHTS = True
config.MODEL.PRETRAINED = ''
config.MODEL.IMAGE_SIZE = [256, 256]  # width * height, ex: 192 * 256
config.MODEL.STYLE = 'pytorch'
# DIRECTORY related
config.PATH = edict()
config.PATH.ROOT = 'dnn_project'
config.PATH.DATASET = os.path.join(config.PATH.ROOT, 'dataset')
config.PATH.LOGS = os.path.join(config.PATH.ROOT, 'logs')
config.PATH.OUTPUTS = os.path.join(config.PATH.ROOT, 'outputs')
config.PATH.HISTORY_FILE = os.path.join(config.PATH.LOGS, config.MODEL.NAME + '-hist.pckl')
config.PATH.LATEST_MODEL_FILE = os.path.join(config.PATH.LOGS, config.MODEL.NAME + '-checkpoint-latest.pth.tar')
config.PATH.BEST_MODEL_FILE = os.path.join(config.PATH.LOGS, config.MODEL.NAME + '-checkpoint-best.pth.tar')
# HARDWARE related
config.HARDWARE = edict()
config.HARDWARE.USE_GPU = True if torch.cuda.is_available else False
config.HARDWARE.GPU_NAME = ''
if config.HARDWARE.USE_GPU:
    config.HARDWARE.DEVICE = 'cuda'
    config.HARDWARE.NB_GPUS = torch.cuda.device_count()    
    config.HARDWARE.GPU_NAME = [torch.cuda.get_device_name(i) for i in range(config.HARDWARE.NB_GPUS)]
else:
    config.HARDWARE.DEVICE = 'cpu'
    
config.HARDWARE.NUM_WORKERS = 1
# DATASET related
config.DATASET = edict()
config.DATASET.ROOT = config.PATH.DATASET
config.DATASET.DATASET = 'mpii'
config.DATASET.TRAIN_SET = 'train'
config.DATASET.TEST_SET = 'valid'
config.DATASET.DATA_FORMAT = 'jpg'
config.DATASET.SELECT_DATA = False
# TRAIN related
config.TRAIN = edict()
config.TRAIN.RETRAIN = True # Option to continue training with pre-trained model
config.TRAIN.LR_FACTOR = 0.1
config.TRAIN.LR_STEP = [90, 110]
config.TRAIN.LR = 0.001
config.TRAIN.OPTIMIZER = 'adam'
config.TRAIN.MOMENTUM = 0.9
config.TRAIN.WD = 0.0001
config.TRAIN.NESTEROV = False
config.TRAIN.GAMMA1 = 0.99
config.TRAIN.GAMMA2 = 0.0
config.TRAIN.NB_EPOCH = 10
config.TRAIN.BEGIN_EPOCH = 0
config.TRAIN.END_EPOCH = 10
config.TRAIN.RESUME = False
config.TRAIN.CHECKPOINT = ''
config.TRAIN.BATCH_SIZE = 32
config.TRAIN.SHUFFLE = True
config.TRAIN.LOG_INTERVAL = 20 #Log every 1% of the progress
# TEST related
config.TEST = edict()
config.TEST.BATCH_SIZE = 32 # size of images for each device
config.TEST.FLIP_TEST = False # Test Model Epoch
config.TEST.POST_PROCESS = True
config.TEST.SHIFT_HEATMAP = True
config.TEST.USE_GT_BBOX = False

# Project setup

In [151]:
# Create project dirs
def makedirs(*inputDirs):
    nb_idx = len(inputDirs)
    for idx, thisdir in enumerate(inputDirs):
        if not os.path.exists(thisdir):
            print('Making Directory: {}...Progress...: {:.1f}%'.format(thisdir, 100*(idx+1)/nb_idx))
            os.makedirs(thisdir)
        else:
            print('Directory Exists: {}...Progress...: {:.1f}%'.format(thisdir, 100*(idx+1)/nb_idx))
# for p in list(config.PATH.values()): makedirs(p)
for p in list(config.PATH.values()):
    if os.path.isdir(p): makedirs(p)
# Logging
logger = logging.getLogger()
stream_hdl = logging.StreamHandler(sys.stdout)
file_hdl = logging.FileHandler(os.path.join(config.PATH.LOGS, config.MODEL.NAME + '-logging.log'), mode = 'a')
formatter = logging.Formatter('%(asctime)s | %(filename)s - %(levelname)s - %(message)s',
                             datefmt='%Y%m%d-%I:%M')
stream_hdl.setFormatter(formatter)
logger.addHandler(stream_hdl)
file_hdl.setFormatter(formatter)
logger.addHandler(file_hdl)
logger.setLevel(logging.INFO)
# Only keep one logger
for h in logger.handlers[:-2]: 
    logger.removeHandler(h)
logging.info('Logging information for project: {}'.format(config.PATH.ROOT))
logging.info('Hardware information:')
logging.info('\t Number of workers: {}'.format(config.HARDWARE.NUM_WORKERS))
logging.info('\t Use GPU: {}'.format(config.HARDWARE.USE_GPU))
if config.HARDWARE.USE_GPU:
    logging.info('\t Number of GPUs: {}'.format(config.HARDWARE.NB_GPUS))
    logging.info('\t GPU list: {}'.format(config.HARDWARE.GPU_NAME))

Directory Exists: dnn_project...Progress...: 100.0%
Directory Exists: dnn_project\dataset...Progress...: 100.0%
Directory Exists: dnn_project\logs...Progress...: 100.0%
Directory Exists: dnn_project\outputs...Progress...: 100.0%
20190926-02:27 | <ipython-input-151-4ec1f13d902c> - INFO - Logging information for project: dnn_project
20190926-02:27 | <ipython-input-151-4ec1f13d902c> - INFO - Hardware information:
20190926-02:27 | <ipython-input-151-4ec1f13d902c> - INFO - 	 Number of workers: 1
20190926-02:27 | <ipython-input-151-4ec1f13d902c> - INFO - 	 Use GPU: True
20190926-02:27 | <ipython-input-151-4ec1f13d902c> - INFO - 	 Number of GPUs: 2
20190926-02:27 | <ipython-input-151-4ec1f13d902c> - INFO - 	 GPU list: ['GeForce GTX 1060 6GB', 'GeForce GTX 1060 6GB']


# Utils

In [146]:
def get_varargin(kwargs, inputkey, defaultValue):
    outputVal = defaultValue
    for key, value in kwargs.items():
        if key == inputkey:
            outputVal = value
        else:
            pass
    return outputVal
# Decorator
def timeit(method):
    def timed(*args, **kwargs):
        start_time = time.time()
        result = method(*args, **kwargs)
        elapsed_time = (time.time() - start_time)*1000
        msg = 'DONE: {func_name}.\t' \
            'Elapsed Time: {elapsed_time:.2f}ms\t'.format(
            func_name = method.__name__,
            elapsed_time = elapsed_time)
        logging.info(msg)
        return result
    return timed
# 
def convert_elapsedTime(elapsed_time):
    '''
    Convert elaped_time to hours, mins, secs
    '''
    hours, rem = divmod(elapsed_time, 3600)
    minutes, seconds = divmod(rem, 60)
    hms = edict()
    hms.hours = hours
    hms.mins = minutes
    hms.secs = seconds
    return hms
# =============================================================================
# Update history
from collections import defaultdict
def update_history(dict_list):
    """Combine a list of history
    history = update_history([history, currentHist.history])
    Returns:
        [type]: [description]
    """
    dd = defaultdict(list)    
    for d in dict_list:
        for key, value in d.items():
            if not hasattr(value, '__iter__'):
                value = (value,)
            [dd[key].append(v) for v in value]    
    return edict(dd)
# =============================================================================
@timeit
def saveHistory(histFile, histInput):   
    """Save training history to file
    
    Args:
        histFile ([type]): [description]
        histInput ([type]): [description]
    """
    logging.info('Saving history file: {}'.format(histFile))
    with open(histFile, 'wb') as fid:
        pickle.dump(histInput, fid)
# =============================================================================
@timeit
def loadHistory(histInputfile):
    """Load history input file
    Args:
        histInputfile ([type]): [description]
    
    Returns:
        [history]: [description]
    """
    logging.info('Load pre-trained history file: {}'.format(histInputfile))
    with open(histInputfile, 'rb') as fid:
        history = pickle.load(fid)
        return history
# Save and Load model
# =============================================================================
@timeit
def save_checkpoint(model, optimizer, **kwargs):
    """Save model structure to json file and weight to h5 file
    
    Args:
        model: NN model
    """
    save_best = get_varargin(kwargs, 'improved_status', False)
    model_filepath = get_varargin(kwargs, 'file_path', config.PATH.LATEST_MODEL_FILE)
    
    logging.info('Save lastest model to: {}'.format(model_filepath))
    torch.save({'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
               }, model_filepath)
    if save_best is True:
        logging.info('Val_acc improved. Save best model to: {}'.format(config.PATH.BEST_MODEL_FILE))
        torch.save({'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
               }, config.PATH.BEST_MODEL_FILE)
# =============================================================================
@timeit
def load_checkpoint(**kwargs):
    '''Save model structure to json file and weight to h5 file
    
    Args:
        model: NN model
    '''
    model_filepath = get_varargin(kwargs, 'file_path', config.PATH.BEST_MODEL_FILE)
    logging.info('Load checkpoint from: {}'.format(model_filepath))
    # ==== BEGIN ====
    checkpoint = torch.load(model_path, map_location = config.HARDWARE.DEVICE) # e.g., trained on GPU, load on CPU
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    return model, optimizer

# Design Model

In [152]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
model = Net().to(config.HARDWARE.DEVICE)
logging.info('Model Name: {}. Model Summary:'.format(config.MODEL.NAME))
torchmodel_summary.logging_model_summary(model,(1,28,28))

20190926-02:27 | <ipython-input-152-df76d64d4ef5> - INFO - Model Name: DNN_model. Model Summary:
20190926-02:27 | torchmodel_summary.py - INFO - ----------------------------------------------------------------
20190926-02:27 | torchmodel_summary.py - INFO -         Layer (type)               Output Shape         Param #
20190926-02:27 | torchmodel_summary.py - INFO - ================================================================
20190926-02:27 | torchmodel_summary.py - INFO -             Conv2d-1           [-1, 20, 24, 24]             520
20190926-02:27 | torchmodel_summary.py - INFO -             Conv2d-2             [-1, 50, 8, 8]          25,050
20190926-02:27 | torchmodel_summary.py - INFO -             Linear-3                  [-1, 500]         400,500
20190926-02:27 | torchmodel_summary.py - INFO -             Linear-4                   [-1, 10]           5,010
20190926-02:27 | torchmodel_summary.py - INFO - ================================================================
2019

# Prepare Datasets

In [11]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(config.PATH.DATASET, train=True, download = True,
                   transform=transforms.Compose([transforms.ToTensor(),
                                                 transforms.Normalize((0.1307,), (0.3081,))
                                                ])),
    batch_size = config.TRAIN.BATCH_SIZE, 
    shuffle = config.TRAIN.SHUFFLE,
    num_workers = config.HARDWARE.NUM_WORKERS,
    pin_memory = True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(config.PATH.DATASET, train=False, 
                   transform=transforms.Compose([transforms.ToTensor(),
                                                 transforms.Normalize((0.1307,), (0.3081,))
                                                ])),
    batch_size = config.TEST.BATCH_SIZE, shuffle=True)

# Train Model

In [148]:
def train_model(config, model, train_loader, epoch, **kwargs):
    '''
    Train model
    '''
    device = get_varargin(kwargs, 'device', config.HARDWARE.DEVICE)
    default_optimizer = optim.SGD(model.parameters(), lr = config.TRAIN.LR, momentum = config.TRAIN.MOMENTUM)
    optimizer = get_varargin(kwargs, 'optimizer', default_optimizer)
#     Switch model to training mode
    model.train()
#     logging parameters
    nb_loops = len(train_loader)
    start_time = time.time()
    tick_time = time.time()
    for batch_idx, (train_data, target) in enumerate(train_loader):
        train_data, target = train_data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(train_data)
#         Backprop
        loss = F.nll_loss(output, target) # Negative loglikelihood function
        loss.backward()
        optimizer.step()
        # measure elapsed time
        batch_time = time.time() - tick_time
        elapsed_time = convert_elapsedTime(time.time() - start_time)
        tick_time = time.time()
        if (batch_idx+1) % np.floor((config.TRAIN.LOG_INTERVAL*nb_loops/100)) == 0:
            msg = 'Epoch: [{0}][{1}/{2}->{3}%]\t' \
            'Elapsed: {hours:0>2}:{mins:0>2}:{secs:0>2}s \t' \
            'Loss {loss:.4f}\t'.format(
                epoch, batch_idx, nb_loops, int(100*batch_idx/nb_loops),
                hours = int(elapsed_time.hours), mins = int(elapsed_time.mins), secs = int(elapsed_time.secs),
                loss = loss)
            logging.info(msg)  
#   Compute train_loss, train_acc
    model.eval()
    train_loss = 0
    train_acc = 0
    with torch.no_grad():
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            train_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            train_acc += pred.eq(target.view_as(pred)).sum().item()
    train_loss /= len(train_loader.dataset)
    train_acc /= len(train_loader.dataset)
#     Logging info
    msg = 'Epoch: [{0}]\t' \
            'loss: {loss:.4f}\t'\
            'acc: {acc:.3f}\t'.format(
                epoch, 
                loss = train_loss,
                acc = train_acc)
    logging.info(msg)
    return train_loss, train_acc

# Test Model

In [101]:
def test_model(config, model, test_loader, epoch, **kwargs):
    """
    Evaluate model
    """
    device = get_varargin(kwargs, 'device', config.HARDWARE.DEVICE)
    model.eval()
    val_loss = 0
    val_acc = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            val_acc += pred.eq(target.view_as(pred)).sum().item()
    val_loss /= len(test_loader.dataset)
    val_acc /= len(test_loader.dataset)
    msg = 'Epoch: [{0}]\t' \
            'val_loss: {loss:.4f}\t'\
            'val_acc: {acc:.3f}\t'.format(
                epoch, 
                loss = val_loss,
                acc = val_acc)
    logging.info(msg)
    return val_loss, val_acc

In [153]:
re_train = False
# re_train = config.TRAIN.RETRAIN
if re_train is True and os.path.isfile(config.PATH.HISTORY_FILE):
    logging.info('Load pre-trained model')
    model, optimizer = load_checkpoint()
    history = loadHistory(config.PATH.HISTORY_FILE)
else:
    logging.info('Start training model from scratch')
    optimizer = optim.SGD(model.parameters(), lr = config.TRAIN.LR, momentum = config.TRAIN.MOMENTUM)
    history = edict()      
start_time = time.time()
for epoch in range(config.TRAIN.NB_EPOCH):
    current_history = edict()
    loss, acc = train_model(config, model, train_loader, epoch, optimizer = optimizer)
    val_loss, val_acc = test_model(config, model, test_loader, epoch)
    current_history.loss = loss
    current_history.acc = acc
    current_history.val_loss = val_loss
    current_history.val_acc = val_acc
    history = update_history([history, current_history])
    improved_status = True
    if (len(history.val_acc) > 1) and (history.val_acc[-1] < history.val_acc[-2]):
        improved_status = False
    saveHistory(config.PATH.HISTORY_FILE, history)
    #   Save checkpoint
    save_checkpoint(model, optimizer, improved_status = improved_status)
#     Log progress
    elapsed_time = convert_elapsedTime(time.time() - start_time)
    msg = 'Epoch:[{0}/{1}]\t' \
    'Elapsed: {hours:0>2}:{mins:0>2}:{secs:0>2}s \t' \
    'loss: {loss:.4f}\t' \
    'acc: {acc:.3f}\t' \
    'val_loss: {val_loss:.4f}\t' \
    'val_acc: {val_acc:.3f}\t'.format(
        epoch, config.TRAIN.NB_EPOCH-1,
        hours = int(elapsed_time.hours), mins = int(elapsed_time.mins), secs = int(elapsed_time.secs),
        loss = loss, acc = acc,
        val_loss = val_loss, val_acc = val_acc)
    logging.info(msg)

20190926-02:28 | <ipython-input-153-ba153d282cbf> - INFO - Start training model from scratch
20190926-02:28 | <ipython-input-148-514a278739d1> - INFO - Epoch: [0][374/1875->19%]	Elapsed: 00:00:03s 	Loss 0.1771	
20190926-02:28 | <ipython-input-148-514a278739d1> - INFO - Epoch: [0][749/1875->39%]	Elapsed: 00:00:06s 	Loss 0.6203	
20190926-02:28 | <ipython-input-148-514a278739d1> - INFO - Epoch: [0][1124/1875->59%]	Elapsed: 00:00:09s 	Loss 0.1707	
20190926-02:28 | <ipython-input-148-514a278739d1> - INFO - Epoch: [0][1499/1875->79%]	Elapsed: 00:00:12s 	Loss 0.2101	
20190926-02:28 | <ipython-input-148-514a278739d1> - INFO - Epoch: [0][1874/1875->99%]	Elapsed: 00:00:15s 	Loss 0.0412	
20190926-02:28 | <ipython-input-148-514a278739d1> - INFO - Epoch: [0]	loss: 0.1081	acc: 0.968	
20190926-02:28 | <ipython-input-101-1a0a3b37080d> - INFO - Epoch: [0]	val_loss: 0.0989	val_acc: 0.971	
20190926-02:28 | <ipython-input-146-6e8ec0621bb5> - INFO - Saving history file: dnn_project\logs\DNN_model-hist.pckl

In [155]:
from GPUtil import showUtilization as gpu_usage

In [156]:
gpu_usage()

| ID | GPU | MEM |
------------------
|  0 | 12% | 25% |
|  1 |  0% |  4% |


In [157]:
logging.info(gpu_usage())

| ID | GPU | MEM |
------------------
|  0 | 13% | 25% |
|  1 |  0% |  4% |
20190926-02:35 | <ipython-input-157-873df15a60d9> - INFO - None


# Validate Model

In [6]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    
def main(**kwargs):
    # Training settings
#     parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
#     parser.add_argument('--batch-size', type=int, default=64, metavar='N',
#                         help='input batch size for training (default: 64)')
#     parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
#                         help='input batch size for testing (default: 1000)')
#     parser.add_argument('--epochs', type=int, default=10, metavar='N',
#                         help='number of epochs to train (default: 10)')
#     parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
#                         help='learning rate (default: 0.01)')
#     parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
#                         help='SGD momentum (default: 0.5)')
#     parser.add_argument('--no-cuda', action='store_true', default=False,
#                         help='disables CUDA training')
#     parser.add_argument('--seed', type=int, default=1, metavar='S',
#                         help='random seed (default: 1)')
#     parser.add_argument('--log-interval', type=int, default=10, metavar='N',
#                         help='how many batches to wait before logging training status')
    
#     parser.add_argument('--save-model', action='store_true', default=False,
#                         help='For Saving the current Model')
#     args = parser.parse_args()
    args = edict()
    args.batch_size = get_varargin(kwargs, 'batch-size', 64)
    args.test_batch_size = get_varargin(kwargs, 'test-batch-size', 1000)
    args.epochs = get_varargin(kwargs, 'epochs', 10)
    args.lr = get_varargin(kwargs, 'lr', 0.01)
    args.momentum = get_varargin(kwargs, 'momentum', 0.5)
    args.no_cuda = get_varargin(kwargs, 'no-cuda', True)
    args.seed = get_varargin(kwargs, 'seed', 1)
    args.log_interval = get_varargin(kwargs, 'log-interval', 100)
    args.save_model = get_varargin(kwargs, 'save-model', False)
    
    use_cuda = not args.no_cuda and torch.cuda.is_available()

    torch.manual_seed(args.seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    

    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader)

    if (args.save_model):
        torch.save(model.state_dict(),"mnist_cnn.pt")

In [8]:
main()



0it [00:00, ?it/s]

  0%|                                                                                      | 0/9912422 [00:00<?, ?it/s]

  0%|▏                                                                     | 32768/9912422 [00:00<00:48, 202059.95it/s]

  1%|▍                                                                     | 57344/9912422 [00:00<00:49, 199949.51it/s]

  1%|▋                                                                     | 90112/9912422 [00:00<00:53, 184315.86it/s]

  1%|▊                                                                    | 122880/9912422 [00:00<00:49, 196262.19it/s]

  2%|█                                                                    | 155648/9912422 [00:00<00:50, 194474.67it/s]

  2%|█▎                                                                   | 180224/9912422 [00:01<00:53, 182495.16it/s]

  2%|█▍                                                                   | 212992/9912422 [00:01<00:50, 191995.73it/s]

  2%|█▋   

Extracting ../data\MNIST\raw\train-images-idx3-ubyte.gz





0it [00:00, ?it/s]


  0%|                                                                                        | 0/28881 [00:00<?, ?it/s]


32768it [00:00, 240937.10it/s]                                                                                         

Extracting ../data\MNIST\raw\train-labels-idx1-ubyte.gz





0it [00:00, ?it/s]


  0%|                                                                                      | 0/1648877 [00:00<?, ?it/s]


  2%|█▍                                                                    | 32768/1648877 [00:00<00:08, 196215.78it/s]


  3%|██▍                                                                   | 57344/1648877 [00:00<00:08, 189519.73it/s]


  5%|███▊                                                                  | 90112/1648877 [00:00<00:07, 198394.17it/s]


  7%|█████▏                                                               | 122880/1648877 [00:00<00:07, 207937.82it/s]


 10%|██████▊                                                              | 163840/1648877 [00:00<00:06, 229130.98it/s]


 12%|████████▌                                                            | 204800/1648877 [00:01<00:05, 246965.85it/s]


 15%|██████████▎                                                          | 245760/1648877 [00:01<00:07, 183022.88it/s]




...



9920512it [01:33, 190547.83it/s]                                                                                       

Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.438883





1654784it [00:24, 171434.11it/s]                                                                                       

Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.367361
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.326299
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.224663
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.339628
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.063072
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.117550
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.143387
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.117899

Test set: Average loss: 0.1018, Accuracy: 9664/10000 (97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.145622
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.100621
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.058878
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.064911
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.121007
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.033273
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.033599
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.038979
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.029846
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.037919

Test set: Average loss: 0.0612, Accuracy: 98

KeyboardInterrupt: 